<a href="https://colab.research.google.com/github/nabussel/User-Recommendation-System/blob/main/UserRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Loading and Exploration:

In [ ]:
import pandas as pd
import numpy as np
import math
columns = ['userid','movieid','rating', 'time']
df = pd.read_csv('data.txt', sep = "\t", names = columns)
df.head()

,userid,movieid,rating,time
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


Data Preprocessing:

In [ ]:
df = df.iloc[:, :3]
df = df.dropna()
pivot_table = df.pivot(index='userid', columns='movieid', values='rating')

Data Split:

In [ ]:
matrixcopy = pivot_table.copy()
np.random.seed(351)
testusers = np.random.choice(matrixcopy.index, size=int(matrixcopy.shape[0]*.2), replace=False)
testvalues = []
for user in testusers:
  indexlist= matrixcopy.iloc[user-1].dropna().index.tolist()
  rating = np.random.choice(indexlist, size = int(len(indexlist)*.2), replace = False)
  matrixcopy.iloc[user-1,rating-1] = np.nan
  for i in rating:
    testvalues.append((user-1, i-1))

User-Based Collaborative filtering:

In [ ]:
def cosine_similarity(user1, user2):
  dotprod = np.dot(user1, user2)
  mag1,mag2 = math.sqrt(np.dot(user1, user1)),math.sqrt(np.dot(user2, user2))
  if mag1 == 0 or mag2 == 0:
      return 0
  return dotprod/(mag1*mag2)

def knn(userid, matrix, k):
  sim = []
  user_rating = matrix.iloc[userid].fillna(0).values
  for i, user in matrix.iterrows():#starts at 1
    if i-1 != userid:
      other_rating = user.fillna(0).values
      similarity = cosine_similarity(user_rating, other_rating)
      sim.append((i-1,similarity))
  sim.sort(key = lambda x: x[1], reverse = True)
  return sim[:k]

def predict(userid,movieid, matrix, k):
  num = 0
  denom = 0
  neighbors = knn(userid, matrix, k)
  for user, cosim in neighbors:
    nrate = matrix.iloc[user, movieid]
    if not np.isnan(nrate):
      num += cosim*nrate
      denom += cosim
  if denom == 0:
    if 5>=matrix.iloc[:, movieid].mean() >=0:
        return matrix.iloc[:, movieid].mean()
    else:
        return matrix.iloc[userid].mean()
  return num/denom

predictedevals = []
for user, movie in testvalues:
    predicted = predict(user, movie, matrixcopy, 10)
    predictedevals.append((user, movie, predicted))

Evaluation:

In [ ]:
summation = 0
pivot_table
for i,j,k in predictedevals:
    summation += (k-(pivot_table.iloc[i,j]))**2
rmse = math.sqrt(summation/len(predictedevals))
print(rmse)

1.0794318403279912


In [ ]:
#Part 1
columns = ['userid','movieid','rating', 'time']
df = pd.read_csv('data.txt', sep = "\t", names = columns)
df.head(0)

#Part2
df = df.iloc[:, :3]
df = df.dropna()
pivot_table = df.pivot(columns='userid', index='movieid', values='rating')

#Part 3
matrixcopy = pivot_table.copy()
np.random.seed(351)
testusers = np.random.choice(matrixcopy.index, size=int(matrixcopy.shape[0]*.2), replace=False)
testvalues = []
for user in testusers:
  indexlist= matrixcopy.iloc[user-1].dropna().index.tolist()
  rating = np.random.choice(indexlist, size = int(len(indexlist)*.2), replace = False)
  matrixcopy.iloc[user-1,rating-1] = np.nan
  for i in rating:
    testvalues.append((user-1, i-1))

#Part 4

def cosine_similarity(user1, user2):
  dotprod = np.dot(user1, user2)
  mag1,mag2 = math.sqrt(np.dot(user1, user1)),math.sqrt(np.dot(user2, user2))
  if mag1 == 0 or mag2 == 0:
      return 0
  return dotprod/(mag1*mag2)

def knn(userid, matrix, k):
  sim = []
  user_rating = matrix.iloc[userid].fillna(0).values
  for i, user in matrix.iterrows():#starts at 1
    if i-1 != userid:
      other_rating = user.fillna(0).values
      similarity = cosine_similarity(user_rating, other_rating)
      sim.append((i-1,similarity))
  sim.sort(key = lambda x: x[1], reverse = True)
  return sim[:k]

def predict(userid,movieid, matrix, k):
  num = 0
  denom = 0
  neighbors = knn(userid, matrix, k)
  for user, cosim in neighbors:
    nrate = matrix.iloc[user, movieid]
    if not np.isnan(nrate):
      num += cosim*nrate
      denom += cosim
  if denom == 0:
    if 5>=matrix.iloc[userid].mean() >=0:
        return matrix.iloc[userid].mean()
    else:
        return matrix.iloc[:, movieid].mean()
  return num/denom


predictedevals = []
for user, movie in testvalues:
    predicted = predict(user, movie, matrixcopy, 10)
    predictedevals.append((user, movie, predicted))

#part 5

summation = 0
pivot_table
for i,j,k in predictedevals:
    summation += (k-(pivot_table.iloc[i,j]))**2
rmse = math.sqrt(summation/len(predictedevals))
print(rmse)

0.9703210431446239
